In [20]:
import pandas as pd
import json
import textwrap

In [9]:
result_path = '/workspaces/seed/paper/weiChainThoughtPrompting2022/output/20230117/gsm8k/flan-t5-xxl/result.json'
failed_path = '/workspaces/seed/paper/weiChainThoughtPrompting2022/output/20230117/gsm8k/flan-t5-xxl/failed.json'

result = json.load(open(result_path))
result

{'checkpoint': 'google/flan-t5-xxl',
 'accuracy': 0.16224412433661864,
 'total': 1319,
 'failure': 1105,
 'regex_parsing_failure': 22}

In [11]:
df = pd.read_json(failed_path)
df.head()

,question,answer,label,output_text,prediction
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18,Janet eats 3 eggs for breakfast and 4 for muff...,24
1,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000,"The house was worth 80,000 * 150% = 140,000 af...",8
2,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20,Wendi feeds her flock of chickens 3 x 20 = 60 ...,40
3,Kylar went to the store to buy glasses for his...,The discount price of one glass is 60/100 * 5 ...,64,Every second glass costs $5 x 60 / 100 = $3. S...,40
4,Toulouse has twice as many sheep as Charleston...,"If Seattle has 20 sheep, Charleston has 4 * 20...",260,Toulouse has twice as many sheep as Charleston...,100


In [35]:
def parsing(instance):
    """
    instance: {'question', 'answer', 'label', 'output_text', 'prediction'}
    """
    template = f"""\
    Q: {instance['question']}
    A: {instance['answer']}
    L: {instance['label']}
    --------------------
    CoT: {instance['output_text']}
    pred: {instance['prediction']}
    """
    return template


In [39]:
# apply parsing function to each row
df['parsing'] = df.apply(parsing, axis=1)
for inst in df.parsing[:3]:
    print(inst)

    Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
    A: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
    L: 18
    --------------------
    CoT: Janet eats 3 eggs for breakfast and 4 for muffins so she has 16 - 3 - 4 = 12 eggs left. She sells them for $2 each so she makes 12 * 2 = $24. The answer is 24.
    pred: 24
    
    Q: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
    A: The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000
He increased the value of the house by 80,000*1.5=<<80000*1.5=1200

Semantic errors dominated. Small mistakes here and there.  

This is `davinci` in zero-shot:
> Q: Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens? 
> 
> Wendi needs to give her chickens 20 cups of feed in the final meal of the day.

This is `davinci` CoT in zero-shot:
> Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens? Let's think step by step.
>
> First, we need to figure out how much Wendi is giving her chickens in total each day. She gives 15 cups in the morning and 25 cups in the afternoon, so she is giving a total of 40 cups of feed each day.
>
> Next, we need to figure out how many cups of feed each chicken needs each day. Wendi gives each of her chickens three cups of feed per day, so she needs to give a total of 60 cups of feed for all of her chickens.
>
> Finally, we need to figure out how many cups of feed Wendi needs to give her chickens in the final meal of the day. Since Wendi has already given 40 cups of feed to her chickens, she needs to give an additional 20 cups of feed in the final meal of the day. Thus, Wendi needs to give her chickens 20 cups of feed in the final meal of the day.

This is `davinci` CoT in few-shot:
>Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today? 
>A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6. 
>
>Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot? 
>A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5. 
>
>Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total? 
>A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39. 
>
>Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny? 
>A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8. 
>
>Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now? 
>A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9. 
>
>Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room? 
>A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29. 
>
>Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday? 
>A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33. 
>
>Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left? 
>A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.
>
>Q: Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?
>A: In the morning, Wendi gave the chickens 15 cups of feed. In the afternoon, she gave them 25 cups of feed. So she has already given the chickens 40 cups of feed in total. She needs to give them 20 x 3 = 60 cups of feed in total. So she needs to give them 60 - 40 = 20 cups of feed in the last meal of the day. The answer is 20 cups of feed.